In [1]:
import requests
import json
from tqdm.auto import tqdm
import os

In [2]:
maxPages = 1
rowsPerPage = 100
i = 1

In [3]:
while(i <= maxPages):

    print(f'Página [{i}]')

    response = requests.get('https://www.siga.ms.gov.br/licitacao/api/public/v1/licitacoes/?'+
                        'page='+str(i)+
                        '&rowsPerPage='+str(rowsPerPage)+
                        '&sort=ASC'+
                        '&totalItems=0'+
                        '&size=100&')                           # Faz a requisição
    if response.status_code != 200 : 
        print(f'Falha ao carregar tabela [{i}] - Status Code: {response.status_code}')
        break
    data_table = json.loads(response.content)                   # Transforma em JSON Object
    maxPages = data_table.get('totalPages')                     # Atualiza o número máximo de páginas
    maxPages = data_table.get('totalPages')                     # Atualiza o número máximo de páginas
    numberOfElements = data_table.get('numberOfElements')       # Número de licitações da página atual

    for j in tqdm(range(numberOfElements)):                     # Acessa todos os arquivos da página atual

        id = data_table.get('content')[j].get('id')
        response = requests.get('https://www.siga.ms.gov.br/licitacao/api/public/v1/licitacoes/'+str(id)+'/anexos')
        if response.status_code != 200:
            print(f'Falha ao carregar item [{str(id)}] - Status Code: {response.status_code}')
        
        if data_table.get('content')[j].get('linkAcessoExterno') == None: # Se estiver no SIGA          
            dados_edital = json.loads(response.content)
            numberOfFiles = len(dados_edital.get('anexos'))

            print(f'\tID [{str(id)}]')

            os.makedirs(os.path.join('licitacoes', str(id)))
            for k in range(0, numberOfFiles):                       # Baixa os arquivos da licitação atual
                os.makedirs(os.path.dirname(os.path.join('licitacoes', str(id), dados_edital.get('anexos')[k].get('arquivoNome'))), exist_ok=True)
                file = open(os.path.join('licitacoes', str(id), dados_edital.get('anexos')[k].get('arquivoNome')), 'wb')
                file.write(
                    requests.get('https://www.siga.ms.gov.br/licitacao/hal/public/arquivos?licitacaoId='+str(data_table.get('content')[j].get('id'))+
                                '&documentoVirtualizadoId='+str(dados_edital.get('anexos')[k].get('idVirtualizado'))+
                                '&nomeArquivo='+str(dados_edital.get('anexos')[k].get('arquivoNome'))+
                                '&externo='+str(dados_edital.get('anexos')[k].get('externo'))+
                                '&uri='+str(dados_edital.get('anexos')[k].get('arquivoUri'))+
                                '&thumbnail=false').content
                )
                file.close()
    
    i += 1

Página [1]


  0%|          | 0/100 [00:00<?, ?it/s]

	ID [158]
	ID [159]
	ID [140]
	ID [149]
	ID [157]
	ID [155]
	ID [152]
	ID [134]
	ID [154]
	ID [88]
	ID [150]
	ID [151]
	ID [153]
	ID [148]
	ID [146]
	ID [143]
	ID [142]
	ID [136]
	ID [139]
	ID [130]
	ID [131]
	ID [133]
	ID [129]
	ID [128]
	ID [124]
	ID [122]
	ID [123]
	ID [121]
	ID [119]
	ID [120]
	ID [117]
	ID [115]
	ID [111]
	ID [103]
	ID [109]
	ID [105]
	ID [106]
	ID [102]
	ID [41]
	ID [55]
	ID [104]
	ID [101]
	ID [99]
	ID [100]
	ID [72]
	ID [98]
	ID [90]
	ID [97]
	ID [85]
	ID [73]
Página [2]


  0%|          | 0/88 [00:00<?, ?it/s]

	ID [96]
	ID [67]
	ID [68]
	ID [63]
	ID [66]
	ID [65]


FileNotFoundError: [Errno 2] No such file or directory: 'licitacoes\\65\\AV. DE LICITAÇÃO PE 0003 2023 AGEPEN 037 168 AQUISIÇÃO DE INSUMOS, UTENSÍLIOS, FERRAMENTAS E MÁQUINAS NECESSÁRIOS À EXECUÇÃO DOS CURSOS DE CORTE E COSTURA INDUSTRIAL , MARCENARIA, SERRALHERIA E PANIFICAÇÃO.pdf'